Implementation based on this: https://huggingface.co/ContactDoctor/Bio-Medical-MultiModal-Llama-3-8B-V1

In [5]:
from huggingface_hub import login

login(token="Hugging_face_API")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/mich02/.cache/huggingface/token
Login successful


In [7]:
%pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 1.6 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 4-bit quantization config for efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the model with quantization and device configuration
model = AutoModel.from_pretrained(
    "ContactDoctor/Bio-Medical-MultiModal-Llama-3-8B-V1",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "ContactDoctor/Bio-Medical-MultiModal-Llama-3-8B-V1",
    trust_remote_code=True
)

# Load and process the image
image = Image.open("Path to Your image").convert('RGB')

# Define the query for multimodal input
question = 'Give the modality, organ, analysis, abnormalities (if any), treatment (if abnormalities are present)?'

# Prepare input messages for the chat interface
msgs = [{'role': 'user', 'content': [image, question]}]

# Generate response
res = model.chat(
    image=image,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.95,
    stream=True
)

# Collect and display generated text
generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')


/home/mich02/Downloads/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm